In [ ]:
# Load gradio
import gradio as gr
import pandas as pd

In [ ]:
# Load parameters
from helpers.db_vector_search import load_db
db_path = "../vector-db"
separator = "\n"
document_path = f"{db_path}/recipe_document.txt"
model_name = "sentence-transformers/all-MiniLM-L6-v2"
db = load_db(
    hf_embedding_model_name=model_name,
    document_path=document_path,
    separator=separator
)
print("Successfully loaded database")

In [ ]:
# Confirm loaded database
test = "List chicken recipes"
result = db.similarity_search(test, k=5)
result

In [ ]:
# Load df
indonesian_recipes = pd.read_csv("../csv/final/indonesian_recipes.csv")
indonesian_recipes

In [ ]:
indonesian_recipes.drop("FlavorProfile", inplace=True, axis=1)
indonesian_recipes

In [ ]:
# Create function to do vector search and return results as dataframe
def find_recipe(query, top_k=10) -> pd.DataFrame:
    recommendations = db.similarity_search(query, k=50)
    found_recipes = []
    for recipe in recommendations:
        found_recipes += [recipe.page_content.split(" x$x ")[0]]
    return indonesian_recipes[indonesian_recipes["Title"].isin(found_recipes)].head(top_k)

In [ ]:
find_recipe("eggs meal")

In [ ]:
# Create dummy function to test displays
def get_recipe(query):
    recipes = find_recipe(query)
    recipe_details = []
    line_break = "\n"
    for recipe in recipes.itertuples():
        title = recipe.Title.title()
        descriptions = f"{title}"
        recipe_details.append(descriptions)
    return line_break.join(recipe_details)

In [ ]:
get_recipe("A simple fried eggs meal")

In [ ]:
# Build demo dashboard as preview
with gr.Blocks(
    title="Dashboard Demo",
    theme="soft"
) as demo_dashboard:
    with gr.Column():
        with gr.Column():
            user_input = gr.Textbox(
                label="Find Recipes",
                placeholder="A simple chicken recipe")
            search_button = gr.Button("Search")
        with gr.Row():
            outputs = gr.Textbox(
                label="Recipes",
                lines=15
            )
        search_button.click(
            fn=get_recipe,
            inputs=user_input,
            outputs=outputs,
        )

In [ ]:
# Launch demo dashboard
demo_dashboard.launch()

In [ ]:
demo_dashboard.close()